In [1]:
#1a) Import Required Packages:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score


In [2]:
#1b)&1c) Load Data & find dimension of Data Frame:
Flightfares_df = pd.read_csv('Flightfares.csv')
Flightfares_df
Flightfares_df.shape  

(638, 18)

In [3]:
# 1d) Show the top 5 records of the data frame
Flightfares_df.head()

,S_CODE,S_CITY,E_CODE,E_CITY,COUPON,NEW,VACATION,SW,HI,S_INCOME,E_INCOME,S_POP,E_POP,SLOT,GATE,DISTANCE,PAX,FARE
0,*,Dallas/Fort Worth TX,*,Amarillo TX,1.00,3,No,Yes,5291.99,28637.0,21112.0,3036732,205711,Free,Free,312,7864,64.11
1,*,Atlanta GA,*,Baltimore/Wash Intl MD,1.06,3,No,No,5419.16,26993.0,29838.0,3532657,7145897,Free,Free,576,8820,174.47
2,*,Boston MA,*,Baltimore/Wash Intl MD,1.06,3,No,No,9185.28,30124.0,29838.0,5787293,7145897,Free,Free,364,6452,207.76
3,ORD,Chicago IL,*,Baltimore/Wash Intl MD,1.06,3,No,Yes,2657.35,29260.0,29838.0,7830332,7145897,Controlled,Free,612,25144,85.47
4,MDW,Chicago IL,*,Baltimore/Wash Intl MD,1.06,3,No,Yes,2657.35,29260.0,29838.0,7830332,7145897,Free,Free,612,25144,85.47


In [4]:
#1e)Remove the first four variables of Dataset (i.e., 'S_CODE', 'S_CITY', 'E_CODE', 'E_CITY')
Flightfares_df = Flightfares_df.iloc[:, 4:]

In [5]:
#1f)Get the dimension of Dataset after removing the first four variables 
Flightfares_df.shape 

(638, 14)

In [6]:
#1g)Get the variables’ data types   
Flightfares_df.dtypes

COUPON      float64
NEW           int64
VACATION     object
SW           object
HI          float64
S_INCOME    float64
E_INCOME    float64
S_POP         int64
E_POP         int64
SLOT         object
GATE         object
DISTANCE      int64
PAX           int64
FARE        float64
dtype: object

In [7]:
#Show the list of Numerical Variables and Categorical Variables
data_types = Flightfares_df.dtypes
# Separate variables into numerical and categorical
numerical_vars = data_types[data_types != 'object'].index.tolist()
categorical_vars = data_types[data_types == 'object'].index.tolist()

# Display the data types and variable lists
print("\nData Types:")
print(data_types)

print("\nNumerical Variables:")
print(numerical_vars)

print("\nCategorical Variables:")
print(categorical_vars)



Data Types:
COUPON      float64
NEW           int64
VACATION     object
SW           object
HI          float64
S_INCOME    float64
E_INCOME    float64
S_POP         int64
E_POP         int64
SLOT         object
GATE         object
DISTANCE      int64
PAX           int64
FARE        float64
dtype: object

Numerical Variables:
['COUPON', 'NEW', 'HI', 'S_INCOME', 'E_INCOME', 'S_POP', 'E_POP', 'DISTANCE', 'PAX', 'FARE']

Categorical Variables:
['VACATION', 'SW', 'SLOT', 'GATE']


In [8]:
#1h) Create a new DataFrame for Numerical Variables
numerical_df = Flightfares_df.select_dtypes(include=['number'])
# Display the new DataFrame containing only numerical variables
print("\nDataFrame for Numerical Variables:")
print(numerical_df)


DataFrame for Numerical Variables:
     COUPON  NEW       HI  S_INCOME  E_INCOME    S_POP    E_POP  DISTANCE  \
0      1.00    3  5291.99   28637.0   21112.0  3036732   205711       312   
1      1.06    3  5419.16   26993.0   29838.0  3532657  7145897       576   
2      1.06    3  9185.28   30124.0   29838.0  5787293  7145897       364   
3      1.06    3  2657.35   29260.0   29838.0  7830332  7145897       612   
4      1.06    3  2657.35   29260.0   29838.0  7830332  7145897       612   
..      ...  ...      ...       ...       ...      ...      ...       ...   
633    1.08    3  2216.70   32991.0   37375.0  8621121   991717      1030   
634    1.08    0  2216.70   32991.0   37375.0  8621121   991717      1030   
635    1.17    3  6797.80   27994.0   37375.0  4948339   991717       960   
636    1.28    3  5566.43   31981.0   37375.0  4549784   991717       858   
637    1.28    3  5566.43   31981.0   37375.0  4549784   991717       858   

       PAX    FARE  
0     7864   64.11

In [9]:
#1i) Create a new DataFrame for Categorical Variables
categorical_df = Flightfares_df.select_dtypes(include=['object'])

# Display the new DataFrame containing only categorical variables
print("\nDataFrame for Categorical Variables:")
print(categorical_df)


DataFrame for Categorical Variables:
    VACATION   SW        SLOT         GATE
0         No  Yes        Free         Free
1         No   No        Free         Free
2         No   No        Free         Free
3         No  Yes  Controlled         Free
4         No  Yes        Free         Free
..       ...  ...         ...          ...
633      Yes   No  Controlled         Free
634      Yes   No        Free  Constrained
635      Yes   No        Free         Free
636      Yes   No        Free         Free
637      Yes   No  Controlled         Free

[638 rows x 4 columns]


In [10]:
#1j)For the Numerical Variables:  create a Correlation Table to explore the relationship between Numerical Variables and Response Variable 'FARE'  
numerical_df.corr().round(2)


,COUPON,NEW,HI,S_INCOME,E_INCOME,S_POP,E_POP,DISTANCE,PAX,FARE
COUPON,1.00,0.02,-0.35,-0.09,0.05,-0.11,0.09,0.75,-0.34,0.50
NEW,0.02,1.00,0.05,0.03,0.11,-0.02,0.06,0.08,0.01,0.09
HI,-0.35,0.05,1.00,-0.03,0.08,-0.17,-0.06,-0.31,-0.17,0.03
S_INCOME,-0.09,0.03,-0.03,1.00,-0.14,0.52,-0.27,0.03,0.14,0.21
E_INCOME,0.05,0.11,0.08,-0.14,1.00,-0.14,0.46,0.18,0.26,0.33
S_POP,-0.11,-0.02,-0.17,0.52,-0.14,1.00,-0.28,0.02,0.28,0.15
E_POP,0.09,0.06,-0.06,-0.27,0.46,-0.28,1.00,0.12,0.31,0.29
DISTANCE,0.75,0.08,-0.31,0.03,0.18,0.02,0.12,1.00,-0.10,0.67
PAX,-0.34,0.01,-0.17,0.14,0.26,0.28,0.31,-0.10,1.00,-0.09
FARE,0.50,0.09,0.03,0.21,0.33,0.15,0.29,0.67,-0.09,1.00


In [11]:
#Variable with the Highest Correlation with 'FARE':Distance
#Correlation Value: # Interpretation of correlation:0.67
#This correlation indiactes that as distance increases the fare also increases.
# The correlation value ranges from -1 to 1. 
# - A correlation close to 1 indicates a strong positive relationship (as one variable increases, the other tends to also increase).
# - A correlation close to -1 indicates a strong negative relationship (as one variable increases, the other tends to decrease).
# - A correlation close to 0 indicates a weak or no linear relationship.

In [12]:
#1k) Create a Pivot Table with the average fare for each category

categories = ['VACATION', 'SW', 'SLOT', 'GATE']

for category in categories:
    # Create a Pivot Table for the current category
    pivot_table = Flightfares_df.pivot_table(values='FARE', index=[category], aggfunc='mean')
    
    # Display the Pivot Table
    print(f"\nPivot Table for {category} with Average Fare:")
    print(pivot_table)




Pivot Table for VACATION with Average Fare:
                FARE
VACATION            
No        173.552500
Yes       125.980882

Pivot Table for SW with Average Fare:
           FARE
SW             
No   188.182793
Yes   98.382268

Pivot Table for SLOT with Average Fare:
                  FARE
SLOT                  
Controlled  186.059396
Free        150.825680

Pivot Table for GATE with Average Fare:
                   FARE
GATE                   
Constrained  193.129032
Free         153.095953


In [13]:
#1l)Convert all the Categorical Variables into Dummy Variables.  
categorical_df = pd.get_dummies(categorical_df,prefix_sep='_', drop_first=True)
categorical_df.columns

Index(['VACATION_Yes', 'SW_Yes', 'SLOT_Free', 'GATE_Free'], dtype='object')

In [14]:
#Show the output: show the Data Frame with converted Categorical Variables.  
print(categorical_df)


     VACATION_Yes  SW_Yes  SLOT_Free  GATE_Free
0               0       1          1          1
1               0       0          1          1
2               0       0          1          1
3               0       1          0          1
4               0       1          1          1
..            ...     ...        ...        ...
633             1       0          0          1
634             1       0          1          0
635             1       0          1          1
636             1       0          1          1
637             1       0          0          1

[638 rows x 4 columns]


In [15]:
#1m)Define Predictors & Outcome Variable  13 Predictors  One Outcome Variable: the average fare on a route (FARE)
predictors = ['VACATION', 'SW', 'SLOT', 'GATE','COUPON', 'NEW', 'HI', 'S_INCOME', 'E_INCOME', 'S_POP', 'E_POP', 'DISTANCE', 'PAX']
outcome = 'FARE'


In [16]:
# Print selected Predictors
Flightfares_df[predictors]



,VACATION,SW,SLOT,GATE,COUPON,NEW,HI,S_INCOME,E_INCOME,S_POP,E_POP,DISTANCE,PAX
0,No,Yes,Free,Free,1.00,3,5291.99,28637.0,21112.0,3036732,205711,312,7864
1,No,No,Free,Free,1.06,3,5419.16,26993.0,29838.0,3532657,7145897,576,8820
2,No,No,Free,Free,1.06,3,9185.28,30124.0,29838.0,5787293,7145897,364,6452
3,No,Yes,Controlled,Free,1.06,3,2657.35,29260.0,29838.0,7830332,7145897,612,25144
4,No,Yes,Free,Free,1.06,3,2657.35,29260.0,29838.0,7830332,7145897,612,25144
...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,Yes,No,Controlled,Free,1.08,3,2216.70,32991.0,37375.0,8621121,991717,1030,34324
634,Yes,No,Free,Constrained,1.08,0,2216.70,32991.0,37375.0,8621121,991717,1030,34324
635,Yes,No,Free,Free,1.17,3,6797.80,27994.0,37375.0,4948339,991717,960,6016
636,Yes,No,Free,Free,1.28,3,5566.43,31981.0,37375.0,4549784,991717,858,4877


In [17]:
# Print Outcome
Flightfares_df[outcome]


0       64.11
1      174.47
2      207.76
3       85.47
4       85.47
        ...  
633    129.63
634    129.63
635    124.87
636    129.62
637    129.62
Name: FARE, Length: 638, dtype: float64

In [18]:
# Convert Categorical Variable into Dummy Variable
X = pd.get_dummies(Flightfares_df[predictors], drop_first=True)
y = Flightfares_df[outcome]


In [19]:
#1n)Partition Data into Training & Validation Sets: Training Set (60%) & Validation Set (40%)
train_X, valid_X, train_y, valid_y = train_test_split(X, y,test_size=0.4, random_state=1)
print('Training set:', train_X.shape, 'Validation set:', valid_X.shape)


Training set: (382, 13) Validation set: (256, 13)


In [20]:
#2a) Fit Regression Model y on X
Flightfares_lm = LinearRegression()
Flightfares_lm.fit(train_X, train_y)


LinearRegression()

In [21]:
# Compute & Print Intercept & Coefficients
print('intercept: ', Flightfares_lm.intercept_)
print('\n', pd.DataFrame({'Predictor': X.columns, 'coefficient': Flightfares_lm.coef_}))


intercept:  52.97284313601136

        Predictor  coefficient
0         COUPON   -11.744039
1            NEW    -2.508229
2             HI     0.006876
3       S_INCOME     0.000626
4       E_INCOME     0.001247
5          S_POP     0.000004
6          E_POP     0.000004
7       DISTANCE     0.077822
8            PAX    -0.000916
9   VACATION_Yes   -35.206099
10        SW_Yes   -42.157140
11     SLOT_Free   -13.533183
12     GATE_Free   -21.185849


In [22]:
#Predict average fare on a route (FARE) using the MLR Model with 13 Predictors
# Predict prices on validation set, print first few predicted values, actual values, and residuals
Flightfares_lm_pred = Flightfares_lm.predict(valid_X)
result = pd.DataFrame({'Predicted': Flightfares_lm_pred,'Actual': valid_y, 'Residual': valid_y - Flightfares_lm_pred})
print(result.head(20)) 

      Predicted  Actual   Residual
101  185.679518  240.88  55.200482
619  305.128677  347.82  42.691323
616  255.240277  279.83  24.589723
446  146.632948  128.36 -18.272948
522  301.481329  289.25 -12.231329
525  231.023180  238.73   7.706820
305  115.379646  140.90  25.520354
41   203.809586  250.73  46.920414
47   184.687411  230.87  46.182589
605  161.471502  164.30   2.828498
455   63.581109  118.17  54.588891
523  314.150454  289.25 -24.900454
339  108.695883  105.41  -3.285883
69   146.045111  113.99 -32.055111
473  276.001471  248.49 -27.511471
460  192.916183  252.97  60.053817
635  165.609858  124.87 -40.739858
23   107.423472  106.77  -0.653472
554   48.449151  106.65  58.200849
13   145.076339  132.05 -13.026339


In [23]:
#Output Table Shows:
#A sample of Predicted Prices & their Residuals (Errors) (relative to Actual Prices) for 20 flights in Validation Set, using MLR Model.


In [24]:
#2b)Compute the performance of the MLR Model on the Training set and Validation set.  
#Compute & Print Performance (Accuracy) Metrics (on Training Set)    
print("\nModel Performance on Training Set:")
regressionSummary(train_y, Flightfares_lm.predict(train_X))



Model Performance on Training Set:

Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 34.7664
            Mean Absolute Error (MAE) : 27.1065
          Mean Percentage Error (MPE) : -4.5089
Mean Absolute Percentage Error (MAPE) : 20.0849


In [25]:
#Compute & Print Performance (Accuracy) Metrics (on Validation Set)
print("\nModel Performance on Validation Set:")
regressionSummary(valid_y, Flightfares_lm_pred)



Model Performance on Validation Set:

Regression statistics

                      Mean Error (ME) : -1.8231
       Root Mean Squared Error (RMSE) : 36.0002
            Mean Absolute Error (MAE) : 28.5002
          Mean Percentage Error (MPE) : -6.6037
Mean Absolute Percentage Error (MAPE) : 21.6048


In [26]:
#2c)Interpretation: 
#Comparing errors for Training Set & Validation Set, shows that Model with 13 Predictors performs well

In [27]:
#3a)Exhaustive Search Method 
def train_model(variables):
 model = LinearRegression()
 model.fit(train_X[list(variables)], train_y)
 return model

def score_model(model, variables):
 pred_y = model.predict(train_X[list(variables)]) 
 return -adjusted_r2_score(train_y, pred_y, model)

allVariables = train_X.columns
results = exhaustive_search(allVariables, train_model, score_model)
data = []
for result in results:
 model = result['model']
 variables = list(result['variables'])
 AIC = AIC_score(train_y, model.predict(train_X[variables]), model)

 d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
 d.update({var: var in result['variables'] for var in allVariables}) 
 data.append(d)

pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables)))


,n,r2adj,AIC,COUPON,DISTANCE,E_INCOME,E_POP,GATE_Free,HI,NEW,PAX,SLOT_Free,SW_Yes,S_INCOME,S_POP,VACATION_Yes
0,1,0.453107,4149.881509,False,True,False,False,False,False,False,False,False,False,False,False,False
1,2,0.605715,4025.892420,False,True,False,False,False,False,False,False,False,True,False,False,False
2,3,0.706909,3913.585125,False,True,False,False,False,False,False,False,False,True,False,False,True
3,4,0.724977,3890.268211,False,True,False,False,False,True,False,False,False,True,False,False,True
4,5,0.737584,3873.328296,False,True,False,False,True,True,False,False,False,True,False,False,True
5,6,0.751947,3852.808698,False,True,False,False,True,True,False,False,True,True,False,False,True
6,7,0.758747,3843.170960,False,True,False,True,False,True,False,True,False,True,False,True,True
7,8,0.765105,3833.945866,False,True,False,True,True,True,False,True,False,True,False,True,True
8,9,0.770266,3826.433471,False,True,False,True,True,True,False,True,True,True,False,True,True
9,10,0.773567,3821.876901,False,True,True,True,True,True,False,True,True,True,False,True,True


In [28]:
#Output of Exhaustive Search:AIC decreases until ten predictors are used and then slowly increases. 
#𝑹_𝒂𝒅𝒋^𝟐 increases until eleven predictors are used and then slowly decreases.
#It indicates that a Model with ten Predictors is the best Model. 
#Dominant Predictor in all Models is distance, with Whether Southwest Airlines or not serves that route  playing important role as well.
#Best Predictors suggested by exhaustive method are 10 variables as listed below:
#1)DISTANCE
#2)SW_Yes
#3)VACATION_Yes
#4)HI
#5)GATE_Free 
#6)SLOT_Free
#7) PAX 
#8)E_POP 
#9)S_POP
#10)E_INCOME

In [29]:
#3b)Code for Backward Elimination – Using AIC
def train_model(variables):
 model = LinearRegression()
 model.fit(train_X[variables], train_y)
 return model

def score_model(model, variables):
 return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print("Best Subset:", best_variables)



Variables: COUPON, NEW, HI, S_INCOME, E_INCOME, S_POP, E_POP, DISTANCE, PAX, VACATION_Yes, SW_Yes, SLOT_Free, GATE_Free
Start: score=3825.24
Step: score=3823.83, remove COUPON
Step: score=3822.80, remove NEW
Step: score=3821.88, remove S_INCOME
Step: score=3821.88, remove None
Best Subset: ['HI', 'E_INCOME', 'S_POP', 'E_POP', 'DISTANCE', 'PAX', 'VACATION_Yes', 'SW_Yes', 'SLOT_Free', 'GATE_Free']


In [30]:
#Interpretation: Best Predictors suggested by Backward Elimination Method are 10 variables as listed below: 
#1)HI 
#2)E_INCOME 
#3)S_POP 
#4)E_POP 
#5)DISTANCE 
#6)PAX
#7)VACATION_Yes 
#8)SW_Yes 
#9)SLOT_Free 
#10)GATE_Free

In [31]:
#Code for Forward Selection – Using AIC 
def train_model(variables):
 if len(variables) == 0:
  return None
 model = LinearRegression()
 model.fit(train_X[variables], train_y)
 return model

def score_model(model, variables):
 if len(variables) == 0:
  return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
 return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print("Best Subset:", best_variables)


Variables: COUPON, NEW, HI, S_INCOME, E_INCOME, S_POP, E_POP, DISTANCE, PAX, VACATION_Yes, SW_Yes, SLOT_Free, GATE_Free
Start: score=4379.42, constant
Step: score=4149.88, add DISTANCE
Step: score=4025.89, add SW_Yes
Step: score=3913.59, add VACATION_Yes
Step: score=3890.27, add HI
Step: score=3873.33, add GATE_Free
Step: score=3852.81, add SLOT_Free
Step: score=3850.30, add PAX
Step: score=3844.58, add E_POP
Step: score=3826.43, add S_POP
Step: score=3821.88, add E_INCOME
Step: score=3821.88, add None
Best Subset: ['DISTANCE', 'SW_Yes', 'VACATION_Yes', 'HI', 'GATE_Free', 'SLOT_Free', 'PAX', 'E_POP', 'S_POP', 'E_INCOME']


In [32]:
#Interpretation: Best Predictors suggested by Forward Selection Method are 10 variables as listed below:
#1)DISTANCE
#2)SW_Yes
#3)VACATION_Yes
#4)HI 
#5)GATE_Free
#6)SLOT_Free
#7)PAX 
#8)E_POP
#9)S_POP
#10)E_INCOME
#We also see “10-predictor” suggested by Forward Selection is identical to “10-predictor” suggested by Exhaustive Search.

In [33]:
#Code for General Stepwise – Using AIC 
def train_model(variables):
 if len(variables) == 0:
  return None
 model = LinearRegression()
 model.fit(train_X[variables], train_y)
 return model

def score_model(model, variables):
 if len(variables) == 0:
  return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
 return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = stepwise_selection(train_X.columns, train_model, score_model, verbose=True)

print("Best Subset:", best_variables)


Variables: COUPON, NEW, HI, S_INCOME, E_INCOME, S_POP, E_POP, DISTANCE, PAX, VACATION_Yes, SW_Yes, SLOT_Free, GATE_Free
Start: score=4379.42, constant
Step: score=4149.88, add DISTANCE
Step: score=4025.89, add SW_Yes
Step: score=3913.59, add VACATION_Yes
Step: score=3890.27, add HI
Step: score=3873.33, add GATE_Free
Step: score=3852.81, add SLOT_Free
Step: score=3850.30, add PAX
Step: score=3844.58, add E_POP
Step: score=3826.43, add S_POP
Step: score=3821.88, add E_INCOME
Step: score=3821.88, unchanged None
Best Subset: ['DISTANCE', 'SW_Yes', 'VACATION_Yes', 'HI', 'GATE_Free', 'SLOT_Free', 'PAX', 'E_POP', 'S_POP', 'E_INCOME']


In [34]:
#Interpretation: Best Predictors suggested by General Stepwise Method are 10 variables as listed below:
#1)DISTANCE
#2)SW_Yes
#3)VACATION_Yes
#4)HI
#5)GATE_Free 
#6)SLOT_Free
#7) PAX 
#8)E_POP 
#9)S_POP
#10)E_INCOME

In [35]:
#3c)comparison table is in the technical report and we can see that all the methods give the same best subset of the predictors as shown in the table in the report. 


In [36]:
#3d) After comparing all the results in the comparison table we can see that all the methods give the Best Subset of Predictors as follows with these predictors:
#1)DISTANCE
#2)SW_Yes
#3)VACATION_Yes
#4)HI
#5)GATE_Free 
#6)SLOT_Free
#7) PAX 
#8)E_POP 
#9)S_POP
#10)E_INCOME
#these are the predictors we suggest for predicting the average fare (FARE) on a new route

In [37]:
#3e) Predictors & Outcome Variable :10 Predictors  One Outcome Variable: the average fare on a route (FARE)
predictors = ['VACATION', 'SW', 'SLOT', 'GATE','HI', 'E_INCOME', 'S_POP', 'E_POP', 'DISTANCE', 'PAX']
outcome = 'FARE'



In [38]:
# Print selected Predictors
Flightfares_df[predictors]


,VACATION,SW,SLOT,GATE,HI,E_INCOME,S_POP,E_POP,DISTANCE,PAX
0,No,Yes,Free,Free,5291.99,21112.0,3036732,205711,312,7864
1,No,No,Free,Free,5419.16,29838.0,3532657,7145897,576,8820
2,No,No,Free,Free,9185.28,29838.0,5787293,7145897,364,6452
3,No,Yes,Controlled,Free,2657.35,29838.0,7830332,7145897,612,25144
4,No,Yes,Free,Free,2657.35,29838.0,7830332,7145897,612,25144
...,...,...,...,...,...,...,...,...,...,...
633,Yes,No,Controlled,Free,2216.70,37375.0,8621121,991717,1030,34324
634,Yes,No,Free,Constrained,2216.70,37375.0,8621121,991717,1030,34324
635,Yes,No,Free,Free,6797.80,37375.0,4948339,991717,960,6016
636,Yes,No,Free,Free,5566.43,37375.0,4549784,991717,858,4877


In [39]:
# Print Outcome
Flightfares_df[outcome]


0       64.11
1      174.47
2      207.76
3       85.47
4       85.47
        ...  
633    129.63
634    129.63
635    124.87
636    129.62
637    129.62
Name: FARE, Length: 638, dtype: float64

In [40]:
# Convert Categorical Variable into Dummy Variable
X = pd.get_dummies(Flightfares_df[predictors], drop_first=True)
y = Flightfares_df[outcome]


In [41]:
# Partition Data into Training & Validation Sets: Training Set (60%) & Validation Set (40%)
train_X, valid_X, train_y, valid_y = train_test_split(X, y,test_size=0.4, random_state=1)
print('Training set:', train_X.shape, 'Validation set:', valid_X.shape)


Training set: (382, 10) Validation set: (256, 10)


In [42]:
#Fit Regression Model y on X
Flightfares_lm = LinearRegression()
Flightfares_lm.fit(train_X, train_y)

LinearRegression()

In [43]:
# Compute & Print Intercept & Coefficients
print('intercept: ', Flightfares_lm.intercept_)
print('\n', pd.DataFrame({'Predictor': X.columns, 'coefficient': Flightfares_lm.coef_}))


intercept:  52.89100015641333

       Predictor  coefficient
0            HI     0.007188
1      E_INCOME     0.001148
2         S_POP     0.000004
3         E_POP     0.000004
4      DISTANCE     0.075558
5           PAX    -0.000829
6  VACATION_Yes   -35.865596
7        SW_Yes   -43.031272
8     SLOT_Free   -13.915304
9     GATE_Free   -21.410803


In [44]:
#interpretation:
#The linear regression equation predicts the airline fare (`Fare`) based on several factors. Among them, the distance (`DISTANCE`) has the highest positive impact, while being a vacation route (`VACATION_Yes`), Southwest Airlines operating on the route (`SW_Yes`), and having no gate constraints (`GATE_Free`) significantly lower the fare. Passenger count (`PAX`) and being a slot-controlled route (`SLOT_Free`) have negative impacts on fare, indicating lower fares with more passengers and absence of slot control. Other factors like household income (`HI`), external income (`E_INCOME`), and population metrics (`S_POP`, `E_POP`) also contribute to fare variations.
# The linear regression equation is Fare=52.89100015641333+HI(0.007188)+ E_INCOME(0.001148)+ S_POP(0.000004) +E_POP(0.000004) + DISTANCE(0.075558)+ VACATION_Yes (-35.865596) + SW_Yes(-13.915304)+ GATE_Free (-21.410803)+PAX (-0.000829)+SLOT_Free (-13.915304)

In [45]:
# Compute & Print Performance (Accuracy) Metrics (on Training Set)
print("\nModel Performance on Training Set:")
regressionSummary(train_y, Flightfares_lm.predict(train_X))



Model Performance on Training Set:

Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 34.8867
            Mean Absolute Error (MAE) : 27.1374
          Mean Percentage Error (MPE) : -4.5313
Mean Absolute Percentage Error (MAPE) : 20.1672


In [46]:
# Compute & Print Performance (Accuracy) Metrics on Validation Set
print("\nModel Performance on Validation Set:")
regressionSummary(valid_y, Flightfares_lm_pred)



Model Performance on Validation Set:

Regression statistics

                      Mean Error (ME) : -1.8231
       Root Mean Squared Error (RMSE) : 36.0002
            Mean Absolute Error (MAE) : 28.5002
          Mean Percentage Error (MPE) : -6.6037
Mean Absolute Percentage Error (MAPE) : 21.6048


In [47]:
#Interpretation: 
#Comparing errors for Training Set & Validation Set, shows that Model with 1O redictors performs well 


In [48]:
# Predict average fare (FARE) using the MLR Model with the Best Subset of Predictors
#Predict prices on validation set, print first few predicted values, actual values, and residuals
Flightfares_lm_pred = Flightfares_lm.predict(valid_X)
result = pd.DataFrame({'Predicted': Flightfares_lm_pred,'Actual': valid_y, 'Residual': valid_y - Flightfares_lm_pred})
print(result.head(20)) # shows output for 20 records
# Calculate average fare
average_fare = result['Actual'].mean()
# Print average fare
print(f'Average Fare: {average_fare}')

      Predicted  Actual   Residual
101  186.221053  240.88  54.658947
619  297.474000  347.82  50.346000
616  259.043514  279.83  20.786486
446  147.916775  128.36 -19.556775
522  299.900759  289.25 -10.650759
525  236.825563  238.73   1.904437
305  118.609256  140.90  22.290744
41   204.533950  250.73  46.196050
47   182.715974  230.87  48.154026
605  166.382759  164.30  -2.082759
455   67.482013  118.17  50.687987
523  307.396257  289.25 -18.146257
339  109.425201  105.41  -4.015201
69   146.902974  113.99 -32.912974
473  275.971452  248.49 -27.481452
460  192.484033  252.97  60.485967
635  165.466655  124.87 -40.596655
23   106.379569  106.77   0.390431
554   52.802674  106.65  53.847326
13   146.489457  132.05 -14.439457
Average Fare: 160.7470703125


In [49]:
#Output Table Shows:
#A sample of Predicted Prices & their Residuals (Errors) (relative to Actual Prices) and average fare for 20 flights in Validation Set, using MLR Model.


In [50]:
#3f) MLR  Equation for predicting average fare (FARE) based on the final MLR Model with the Best Subset of Predictors.
#Fare=52.89100015641333+HI(0.007188)+ E_INCOME(0.001148)+ S_POP(0.000004) +E_POP(0.000004) + DISTANCE(0.075558)+ VACATION_Yes (-35.865596) + SW_Yes(-13.915304)+ GATE_Free (-21.410803)+PAX (-0.000829)+SLOT_Free (-13.915304)

In [51]:
# Given coefficients from the MLR model
coefficients = {
    'HI': 0.007188,
    'E_INCOME': 0.001148,
    'S_POP': 0.000004,
    'E_POP': 0.000004,
    'DISTANCE': 0.075558,
    'VACATION_Yes': -35.865596,
    'SW_Yes': -13.915304,
    'GATE_Free': -21.410803,
    'PAX': -0.000829,
    'SLOT_Free': -13.915304
}

# Intercept from the MLR model
intercept = 52.89100015641333

# Function to calculate predicted fare
def predict_fare(inputs):
    fare = intercept
    for predictor, coefficient in coefficients.items():
        fare += coefficient * inputs[predictor]
    return fare


# Input values
input_values = {
    'COUPON': 1.202,
    'NEW': 3,
    'VACATION_Yes': 0,  # 0 for 'No', 1 for 'Yes'
    'SW_Yes': 0,  # 0 for 'No', 1 for 'Yes'
    'HI': 4442.141,
    'S_INCOME': 28760,
    'E_INCOME': 27664,
    'S_POP': 4557004,
    'E_POP': 3195503,
    'SLOT_Free': 1,  # 1 for 'Free', 0 for 'Not Free'
    'GATE_Free': 1,  # 1 for 'Free', 0 for 'Not Free'
    'PAX': 12782,
    'DISTANCE': 1976
}
# Predict the fare
predicted_fare = predict_fare(input_values)

print("Predicted Fare:", predicted_fare)

#3g)COUPON = 1.202, NEW = 3, VACATION = No, SW = No, HI = 4442.141, S_INCOME = 28,760,E_INCOME = 27,664, S_POP = 4,557,004, E_POP = 3,195,503, SLOT = Free, GATE = Free, PAX = 12,782, DISTANCE = 1976 miles
# substituting in mlr equation we get Fare=52.89100015641333+(4442.141×0.007188)+(27764×0.001148)+(4557004×0.000004)+(3195503×0.000004)+(1976×0.075558)+0×(−35.865596))+(0×(−13.915304))+1×(−21.410803))+(12782×−0.000829)+(1×(−13.915304))
#Fare=250.96963266441333

Predicted Fare: 250.96963266441333


In [52]:
# Given coefficients from the MLR model
coefficients = {
    'HI': 0.007188,
    'E_INCOME': 0.001148,
    'S_POP': 0.000004,
    'E_POP': 0.000004,
    'DISTANCE': 0.075558,
    'VACATION_Yes': -35.865596,
    'SW_Yes': -13.915304,
    'GATE_Free': -21.410803,
    'PAX': -0.000829,
    'SLOT_Free': -13.915304
}

# Intercept from the MLR model
intercept = 52.89100015641333

# Function to calculate predicted fare
def predict_fare(inputs):
    fare = intercept
    for predictor, coefficient in coefficients.items():
        fare += coefficient * inputs[predictor]
    return fare


# Input values
input_values = {
    'COUPON': 1.202,
    'NEW': 3,
    'VACATION_Yes': 0,  # 0 for 'No', 1 for 'Yes'
    'SW_Yes': 1,  # 0 for 'No', 1 for 'Yes'
    'HI': 4442.141,
    'S_INCOME': 28760,
    'E_INCOME': 27664,
    'S_POP': 4557004,
    'E_POP': 3195503,
    'SLOT_Free': 1,  # 1 for 'Free', 0 for 'Not Free'
    'GATE_Free': 1,  # 1 for 'Free', 0 for 'Not Free'
    'PAX': 12782,
    'DISTANCE': 1976
}
# Predict the fare
predicted_fare = predict_fare(input_values)

print("Predicted Fare:", predicted_fare)
#3h)COUPON = 1.202, NEW = 3, VACATION = No, SW = Yes, HI = 4442.141, S_INCOME = 28,760,E_INCOME = 27,664, S_POP = 4,557,004, E_POP = 3,195,503, SLOT = Free, GATE = Free, PAX = 12,782, DISTANCE = 1976 miles
#substituting in mlr equation we get Fare=52.89100015641333+(4442.141×0.007188)+(27764×0.001148)+(4557004×0.000004)+(3195503×0.000004)+(1976×0.075558)+(0×(−35.865596))+(1×(−13.915304))+(1×(−21.410803))+(12782×−0.000829)+(1×(−13.915304)
#Fare=237.0543286644133

Predicted Fare: 237.0543286644133


In [53]:
#Using the final MLR Model with the Best Subset of Predictors to predict the average fare on a
#route if Southwest decides to cover the route in part g.The reduction  in average fare on the route if southwest operates is 13.915304 